# DR vs GBR: Distributional Repair & Group-Blind Repair (Adult dataset)

This notebook compares two dataset-repair approaches:

- **Distributional Repair (DR)** group-aware: learns two OT maps (S=0 and S=1) to a shared barycenter (conditional on U).

- **GroupBlind Repair (GBR)** group-blind: learns one OT map from the pooled distribution to a target (we will align this target with DR’s barycenter so the comparison is meaningful). Variants:

    - `baseline` (no fairness vector)

    - `partial` / `total` (use fairness vector *V*), optional.

**Data:** Adult (AIF360).  
**Protected (S):** sex (0 = Female, 1 = Male)  
**Unprotected (U):** college_educated (0/1)  
**Features (X):** age, hours-per-week (continuous)

We compute U-Mean KLD (average KL divergence between P(X|S=1,U) and P(X|S=0,U), averaged over *U*) on:

- **Research** split (used to learn transport)

- **Archive** split (held-out to check generalization)

Smaller values indicate closer group distributions conditional on *U* (0 = identical).

## Imports & config

In [1]:
import time
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ot

from sklearn.neighbors import KernelDensity
from aif360.datasets import AdultDataset

from humancompatible.repair.distributional_repair import DistributionalRepair
from humancompatible.repair.group_blind_repair import GroupBlindRepair

np.random.seed(0)

## Dataset loader

In [2]:
def load_adult_dataset(s,u,x,y):
    def custom_preprocessing(df):
        pd.set_option('future.no_silent_downcasting', True)
        def group_race(x):
            if x == "White":
                return 1.0
            else:
                return 0.0

        df['race'] = df['race'].apply(lambda x: group_race(x))

        # Encode 'sex' column as numerical values
        df['sex'] = df['sex'].map({'Female': 0.0, 'Male': 1.0})

        df['Income Binary'] = df['income-per-year']
        df['Income Binary'] = df['Income Binary'].replace(to_replace='>50K.', value=1, regex=True)
        df['Income Binary'] = df['Income Binary'].replace(to_replace='>50K', value=1, regex=True)
        df['Income Binary'] = df['Income Binary'].replace(to_replace='<=50K.', value=0, regex=True)
        df['Income Binary'] = df['Income Binary'].replace(to_replace='<=50K', value=0, regex=True)
        # 1 if education-num is greater than 9, 0 otherwise
        df['college_educated'] = (df['education-num'] > 9).astype(int)

        #drop nan columns
        df = df.dropna()

        return df

    adult = AdultDataset(
        label_name=y,
        favorable_classes=[1,1],
        protected_attribute_names=[s],
        privileged_classes=[[1.0]],
        instance_weights_name=None,
        categorical_features=[],
        features_to_keep=[s]+[u]+x,
        na_values=[],
        custom_preprocessing=custom_preprocessing,
        features_to_drop=[],
        metadata={}
    )
    return adult

## KLD (Kullback-Leibler Divergence) evaluation helpers

We use a simple distribution-level score to quantify how much group signal remains in each feature after repair. 

For each value of U, we take the feature X restricted to the two groups (S=0 and S=1), fit 1-D Gaussian KDEs for each group on a shared grid of 500 points spanning the pooled min/max, add a tiny *ε* to avoid zeros, and compute a one-directional Kullback–Leibler divergence KL(P0||P1) (i.e., how well the S=1 density explains S=0). 

We then weight that KL by the prevalence of the corresponding U value and average across all U values. Lower values mean the conditional distributions P(X|S,U) are closer (0 means identical). 

Because KL is asymmetric, the score reflects the fixed direction S=0 -> S=1; we keep this direction consistent across runs so results are comparable. U values with missing group samples are skipped. The absolute scale depends on the KDE bandwidth and grid resolution, so the metric is best used for before/after comparisons rather than as an absolute number.

In [3]:
def _eval_kld(x_0, x_1):
    support = np.linspace(np.min([np.min(x_0), np.min(x_1)]), np.max([np.max(x_0), np.max(x_1)]), 500).reshape(-1,1)
    kde_0 = KernelDensity(kernel='gaussian',bandwidth='silverman').fit(x_0.reshape(-1,1))
    pmf_0 = np.exp(kde_0.score_samples(support)) 
    #add a small value to avoid division by zero
    pmf_0 += 1e-10
    kde_1 = KernelDensity(kernel='gaussian',bandwidth='silverman').fit(x_1.reshape(-1,1))
    pmf_1 = np.exp(kde_1.score_samples(support))
    pmf_1 += 1e-10
    return - np.sum(pmf_0 * np.log(pmf_1 / pmf_0))

def eval_kld(x, s, u, order=[0,1]):
    tot_kld = 0.0
    for u_val, u_count in u.value_counts().items():
        mask_0 = np.asarray((u == u_val) & (s == 0))
        mask_1 = np.asarray((u == u_val) & (s == 1))
        if (np.sum(mask_0) == 0) or (np.sum(mask_1) == 0):
            continue
        tmp = _eval_kld(x[mask_0].values, x[mask_1].values)
        if np.isnan(tmp):
            continue
        tot_kld += tmp * u_count / len(u)
    return tot_kld